# PROMIGAS.CL Price Analysis

This notebook loads `PROMIGAS.CL` from the SQLite database and runs the same analysis set (price, volume, RSI, and Fibonacci by timeframe).

In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
TICKER = "PROMIGAS.CL"
candidate_paths = [Path("stock_data.db"), Path("../stock_data.db")]
db_path = next((p for p in candidate_paths if p.exists()), None)

if db_path is None:
    raise FileNotFoundError("Could not find stock_data.db in current or parent directory.")

db_path

In [ ]:
query = """
SELECT date, open, high, low, close, adj_close, volume, dividends, stock_splits
FROM ticker_prices
WHERE ticker = ?
ORDER BY date
"""

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql_query(query, conn, params=(TICKER,), parse_dates=["date"])

if df.empty:
    raise ValueError(f"No rows found for {TICKER}.")

# Filter to 2018 and later
df = df[df["date"] >= "2018-01-01"].reset_index(drop=True)

df.head()


In [ ]:
df["close_20ma"] = df["close"].rolling(20).mean()
df["close_50ma"] = df["close"].rolling(50).mean()
df["close_200ma"] = df["close"].rolling(200).mean()

# Exponential Moving Averages
df["ema_20"] = df["close"].ewm(span=20, adjust=False).mean()
df["ema_50"] = df["close"].ewm(span=50, adjust=False).mean()

# Bollinger Bands (20-day, 2 std)
_bb_std = df["close"].rolling(20).std()
df["bb_upper"] = df["close_20ma"] + 2 * _bb_std
df["bb_lower"] = df["close_20ma"] - 2 * _bb_std


In [ ]:
# Interactive Plotly chart (Price + Volume + MAs)
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 2}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA', line={'width': 1.5, 'dash': 'dot', 'color': 'gold'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_50ma'], name='50-day MA', line={'width': 1.5, 'color': 'orange'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_200ma'], name='200-day MA', line={'width': 1.5, 'color': 'red'}), secondary_y=False)

fig.add_trace(go.Bar(x=df['date'], y=df['volume'], name='Volume', opacity=0.25), secondary_y=True)

fig.update_layout(
    title=f'{TICKER} Close Price, Volume & Moving Averages (Plotly)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    bargap=0,
    template='plotly_dark',
)
fig.update_yaxes(title_text='Volume', secondary_y=True)
fig.show()


In [ ]:
# RSI calculation (7, 14 and 21)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# RSI 7 — fastest, best for short-term signals
avg_gain_7 = gain.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
avg_loss_7 = loss.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
df['rsi_7'] = 100 - (100 / (1 + avg_gain_7 / avg_loss_7))

# RSI 14 — standard medium-term
avg_gain_14 = gain.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
avg_loss_14 = loss.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
df['rsi_14'] = 100 - (100 / (1 + avg_gain_14 / avg_loss_14))

# RSI 21 — slower, better for position/swing confirmation
avg_gain_21 = gain.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
avg_loss_21 = loss.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
df['rsi_21'] = 100 - (100 / (1 + avg_gain_21 / avg_loss_21))


In [ ]:
# Interactive Plotly chart (RSI 7, 14 & 21)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_7'],  name='RSI 7',  line={'color': '#00CC44', 'width': 1.5, 'dash': 'dot'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_14'], name='RSI 14', line={'color': 'orange',  'width': 1.5}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_21'], name='RSI 21', line={'color': '#FF1111', 'width': 1.5, 'dash': 'dash'}))
fig.add_hline(y=70, line_dash='dash', line_color='red',    annotation_text='Overbought 70', annotation_position='left')
fig.add_hline(y=30, line_dash='dash', line_color='#4444FF', annotation_text='Oversold 30',   annotation_position='left')
fig.update_layout(
    title=f'{TICKER} RSI 7, 14 & 21 (Plotly)',
    xaxis_title='Date',
    yaxis_title='RSI',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.update_yaxes(range=[0, 100])
fig.show()


In [ ]:
# Interactive Plotly chart — EMA 20 & EMA 50
# EMAs react faster than simple MAs; useful for short/medium-term signals.
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'steelblue'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_20'], name='EMA 20', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_50'], name='EMA 50', line={'width': 2.5, 'color': '#FF1111'}))

fig.update_layout(
    title=f'{TICKER} — Exponential Moving Averages (EMA 20 & 50)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


In [ ]:
# Interactive Plotly chart — Bollinger Bands (20-day, ±2 std)
# Upper/Lower bands identify overbought/oversold zones based on volatility.
fig = go.Figure()

# Shaded band (upper → lower fill)
fig.add_trace(go.Scatter(
    x=pd.concat([df['date'], df['date'][::-1]]),
    y=pd.concat([df['bb_upper'], df['bb_lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(180, 180, 255, 0.18)',
    line={'color': 'rgba(0,0,0,0)'},
    name='BB Band',
    showlegend=True,
))

fig.add_trace(go.Scatter(x=df['date'], y=df['bb_upper'], name='BB Upper', line={'width': 2.5, 'color': '#FF1111'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA (Mid)', line={'width': 2, 'dash': 'dot', 'color': '#FFD700'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['bb_lower'], name='BB Lower', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'white'}))

fig.update_layout(
    title=f'{TICKER} — Bollinger Bands (20-day, ±2 std)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


In [ ]:
# MACD (12, 26, 9)
df["ema_12"] = df["close"].ewm(span=12, adjust=False).mean()
df["ema_26"] = df["close"].ewm(span=26, adjust=False).mean()
df["macd_line"] = df["ema_12"] - df["ema_26"]
df["macd_signal"] = df["macd_line"].ewm(span=9, adjust=False).mean()
df["macd_hist"] = df["macd_line"] - df["macd_signal"]


In [ ]:
# Momentum (MACD)
plot_df = df.tail(300).copy()

fig = go.Figure()
fig.add_trace(go.Bar(x=plot_df["date"], y=plot_df["macd_hist"], name="MACD hist", marker_color="#90CAF9", opacity=0.55))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["macd_line"], name="MACD line", line=dict(color="#42A5F5", width=1.7)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["macd_signal"], name="MACD signal", line=dict(color="#EF5350", width=1.4, dash="dot")))
fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.update_layout(
    height=380,
    title=f"{TICKER} Momentum (MACD)",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11)),
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="MACD")
fig.show()


## Volume quick wins (2 charts)

Low-hanging volume signals: Relative Volume (RVOL) and On-Balance Volume (OBV).

In [ ]:
# Volume metrics used by the next 2 charts
# Restrict to last 12 months to avoid stale/nonsensical early volume data
cutoff = pd.Timestamp.now() - pd.DateOffset(years=1)
vol_df = df[df["date"] >= cutoff].copy().reset_index(drop=True)

vol = vol_df["volume"].fillna(0).astype(float)
close = vol_df["close"].astype(float)

vol_df["vol_sma_20"] = vol.rolling(20).mean()
vol_df["rvol_20"] = (vol / vol_df["vol_sma_20"]).replace([float("inf"), float("-inf")], pd.NA)

# OBV increments volume when close rises, subtracts when close falls
delta = close.diff().fillna(0)
direction = delta.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
vol_df["obv"] = (direction * vol).cumsum()
vol_df["obv_ema_20"] = vol_df["obv"].ewm(span=20, adjust=False).mean()

print(f"RVOL and OBV columns ready ({len(vol_df)} rows, last 12 months).")

In [ ]:
# Volume chart 1/2: Relative Volume (RVOL 20)
plot_df = vol_df.copy()

fig = go.Figure()
fig.add_trace(go.Bar(x=plot_df["date"], y=plot_df["rvol_20"], name="RVOL 20", marker_color="#00B8D4", opacity=0.7))
fig.add_hline(y=1.0, line_dash="dash", line_color="white", annotation_text="Baseline (1.0)", annotation_position="top right")
fig.add_hline(y=1.5, line_dash="dot", line_color="orange", annotation_text="Expansion (1.5)", annotation_position="top right")

fig.update_layout(
    height=340,
    title=f"{TICKER} Relative Volume (20) — last 12 months",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    showlegend=False
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="RVOL")
fig.show()

In [ ]:
# Volume chart 2/2: OBV + OBV EMA(20)
plot_df = vol_df.copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["obv"], name="OBV", line=dict(color="#80CBC4", width=1.7)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["obv_ema_20"], name="OBV EMA 20", line=dict(color="#FFAB40", width=1.6, dash="dot")))

fig.update_layout(
    height=340,
    title=f"{TICKER} OBV trend confirmation — last 12 months",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11))
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="OBV")
fig.show()